Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'Data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

First we start with the Logistic Regression model

In [13]:
batch_size = 128
betas = [1e-3, 1e-2, 1e-1, 0, 0.5, 1]
graph = tf.Graph()
for beta in betas:
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(tf_train_dataset, weights) + biases
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) +\
            beta*tf.nn.l2_loss(weights)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
        test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

        num_steps = 3001

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        test_accuracy_sgd = accuracy(test_prediction.eval(), test_labels)
        print("beta: %.4f" % beta)
        print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))

Initialized
beta: 0.0010
Validation accuracy: 81.9%
Initialized
beta: 0.0100
Validation accuracy: 81.4%
Initialized
beta: 0.1000
Validation accuracy: 77.8%
Initialized
beta: 0.0000
Validation accuracy: 78.7%
Initialized
beta: 0.5000
Validation accuracy: 61.5%
Initialized
beta: 1.0000
Validation accuracy: 51.1%


Next we check the validation error for the regularized 1-hidden layer neural net - we will start by only regularizing the output layer.

In [ ]:
batch_size = 128
betas = [1e-3, 1e-2, 1e-1, 0, 0.5, 1]
num_hidden_nodes = 1024
graph = tf.Graph()
for beta in betas:
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights_h1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
        weights_out = tf.Variable(
          tf.truncated_normal([num_hidden_nodes, num_labels]))
        biases_in = tf.Variable(tf.zeros([num_hidden_nodes]))
        biases_out = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(
          tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_in), weights_out) + biases_out
        
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) +\
            beta*tf.nn.l2_loss(weights_out)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(
          tf.matmul(
              tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_in), weights_out) + biases_out
        )
        test_prediction = tf.nn.softmax(
          tf.matmul(
              tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_in), weights_out) + biases_out
        )

    num_steps = 3001
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        test_accuracy_sgd = accuracy(test_prediction.eval(), test_labels)
        print("beta: %.4f" % l_)
        print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))

Best beta:
* LR is **0.001**
* neural net is **0.001**
  
Test accuracy, the neural net contributes by an increase of 1 %:
* LR: **88.9 %**
* Neural net: **89.9 %**

In [ ]:
batch_size = 128
betas = [1e-3]
graph = tf.Graph()
for beta in betas:
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(tf_train_dataset, weights) + biases
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) +\
            beta*tf.nn.l2_loss(weights)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
        test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

        num_steps = 3001

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        test_accuracy_sgd = accuracy(test_prediction.eval(), test_labels)
        print("beta: %.4f" % l_)
        print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
        test_accuracy_sgd = accuracy(test_prediction.eval(), test_labels)
        print("Test accuracy: %.1f%%" % test_accuracy_sgd)

In [ ]:
batch_size = 128
betas = [1e-3]
num_hidden_nodes = 1024
graph = tf.Graph()
for beta in betas:
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights_h1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
        weights_out = tf.Variable(
          tf.truncated_normal([num_hidden_nodes, num_labels]))
        biases_in = tf.Variable(tf.zeros([num_hidden_nodes]))
        biases_out = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(
          tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_in), weights_out) + biases_out
        
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) +\
            beta*tf.nn.l2_loss(weights_out)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(
          tf.matmul(
              tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_in), weights_out) + biases_out
        )
        test_prediction = tf.nn.softmax(
          tf.matmul(
              tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_in), weights_out) + biases_out
        )

    num_steps = 3001
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        test_accuracy_sgd = accuracy(test_prediction.eval(), test_labels)
        print("lambda: %.4f" % l_)
        print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
        test_accuracy_1_hl = accuracy(test_prediction.eval(), test_labels)
        print("Test accuracy: %.1f%%" % test_accuracy_1_hl)

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

We observe that the training procedure travels through and obtains a full score on the batch data quite fast - the performance on the validation and test data is however not increased any further.

In [20]:
batch_size = 128
betas = [1e-3]
num_hidden_nodes = 1024

num_records = 3*batch_size
train_dataset_small = train_dataset[:num_records]
train_labels_small = train_labels[:num_records]

graph = tf.Graph()
for beta in betas:
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights_h1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
        weights_out = tf.Variable(
          tf.truncated_normal([num_hidden_nodes, num_labels]))
        biases_in = tf.Variable(tf.zeros([num_hidden_nodes]))
        biases_out = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(
          tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_in), weights_out) + biases_out
        
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) +\
            beta*tf.nn.l2_loss(weights_out)

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(
          tf.matmul(
              tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_in), weights_out) + biases_out
        )
        test_prediction = tf.nn.softmax(
          tf.matmul(
              tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_in), weights_out) + biases_out
        )

    num_steps = 3001
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset_small[offset:(offset + batch_size), :]
            batch_labels = train_labels_small[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
                    
        print("beta: %.4f" % beta)
        print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
        test_accuracy_1_hl = accuracy(test_prediction.eval(), test_labels)
        print("Test accuracy: %.1f%%" % test_accuracy_1_hl)

Initialized
Minibatch loss at step 0: 313.621124
Minibatch accuracy: 14.1%
Validation accuracy: 22.7%
Minibatch loss at step 500: 2.635849
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 1000: 1.598522
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 1500: 0.969432
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 2000: 0.587918
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 2500: 0.356549
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 3000: 0.216235
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
beta: 0.0010
Validation accuracy: 70.1%
Test accuracy: 76.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Best result and hyperparameter setting obtained when the probability of keeping the variable is almost equal to 1:
* keep prob: 0.9000
* beta: 0.0010
* Validation accuracy: 86.3%
* Test accuracy: 93.3%

or when not using any dropout regularization at all:
* keep prob: 1.0000
* beta: 0.0010
* Validation accuracy: 86.4%
* Test accuracy: 93.3%

In [30]:
# run a nn-model when combining l2 and dropout regularization
batch_size = 128
betas = [0, 1e-3, 1e-2, 1e-1, 0, 0.5, 1]
keep_probs = [0.01, 0.1, 0.25, 0.35, 0.4, 0.5, 0.6, 0.75, 0.9, 1]
num_hidden_nodes = 1024
graph = tf.Graph()
for beta in betas:
    for k_p in keep_probs:
        with graph.as_default():

            # Input data. For the training data, we use a placeholder that will be fed
            # at run time with a training minibatch.
            tf_train_dataset = tf.placeholder(tf.float32,
                                            shape=(batch_size, image_size * image_size))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            tf_valid_dataset = tf.constant(valid_dataset)
            tf_test_dataset = tf.constant(test_dataset)
            keep_prob = tf.placeholder(tf.float32)

            # Variables.
            weights_h1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
            weights_out = tf.Variable(
              tf.truncated_normal([num_hidden_nodes, num_labels]))
            biases_in = tf.Variable(tf.zeros([num_hidden_nodes]))
            biases_out = tf.Variable(tf.zeros([num_labels]))

            # Training computation.
            layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_in)
            drop_out = tf.nn.dropout(layer_1, keep_prob=k_p)
            logits = tf.matmul(drop_out, weights_out) + biases_out

            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) +\
                beta*(tf.nn.l2_loss(weights_h1) + tf.nn.l2_loss(weights_out))

            # Optimizer.
            optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

            # Predictions for the training, validation, and test data.
            train_prediction = tf.nn.softmax(logits)
            valid_prediction = tf.nn.softmax(
              tf.matmul(
                  tf.nn.dropout(
                      tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_in)
                      , keep_prob=1)
                  , weights_out) + biases_out
            )
            test_prediction = tf.nn.softmax(
              tf.matmul(
                  tf.nn.dropout(
                      tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_in)
                      , keep_prob=1)
                  , weights_out) + biases_out
            )

        num_steps = 3001
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print("Initialized")
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {tf_train_dataset : batch_data
                             , tf_train_labels : batch_labels
                            }
                _, l, predictions = session.run(
                  [optimizer, loss, train_prediction], feed_dict=feed_dict)
#                 if (step % 500 == 0):
#                     print("Minibatch loss at step %d: %f" % (step, l))
#                     print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#                     print("Validation accuracy: %.1f%%" % accuracy(
#                         valid_prediction.eval(), valid_labels))

            print("\nkeep prob: %.4f" % k_p)
            print("beta: %.4f" % beta)
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            test_accuracy_1_hl = accuracy(test_prediction.eval(), test_labels)
            print("Test accuracy: %.1f%%\n" % test_accuracy_1_hl)

Initialized

keep prob: 0.0100
beta: 0.0000
Validation accuracy: 10.0%
Test accuracy: 10.0%

Initialized

keep prob: 0.1000
beta: 0.0000
Validation accuracy: 71.9%
Test accuracy: 78.9%

Initialized

keep prob: 0.2500
beta: 0.0000
Validation accuracy: 68.5%
Test accuracy: 75.5%

Initialized

keep prob: 0.3500
beta: 0.0000
Validation accuracy: 76.8%
Test accuracy: 84.4%

Initialized

keep prob: 0.4000
beta: 0.0000
Validation accuracy: 78.3%
Test accuracy: 86.2%

Initialized

keep prob: 0.5000
beta: 0.0000
Validation accuracy: 80.5%
Test accuracy: 87.4%

Initialized

keep prob: 0.6000
beta: 0.0000
Validation accuracy: 80.8%
Test accuracy: 88.3%

Initialized

keep prob: 0.7500
beta: 0.0000
Validation accuracy: 81.3%
Test accuracy: 89.2%

Initialized

keep prob: 0.9000
beta: 0.0000
Validation accuracy: 82.1%
Test accuracy: 89.3%

Initialized

keep prob: 1.0000
beta: 0.0000
Validation accuracy: 82.7%
Test accuracy: 89.8%

Initialized

keep prob: 0.0100
beta: 0.0010
Validation accuracy: 10.0%

As for the extreme overfitting case we observe that a fairly high probability of excluding variables obtains the best result, thus we could argue when there is a greater risk of overfitting, then dropout regularization can improve model performance:

* keep prob: 0.2500
* beta: 0.0010
* Validation accuracy: 70.6%
* Test accuracy: 78.2%

In [33]:
# run a nn-model when combining l2 and dropout regularization
batch_size = 128
betas = [1e-3]
num_records = batch_size*2
train_dataset_small = train_dataset[:num_recods]
train_labels_small = train_labels[:num_records]
keep_probs = [0.01, 0.1, 0.25, 0.35, 0.4, 0.5, 0.6, 0.75, 0.9, 1]
num_hidden_nodes = 1024
graph = tf.Graph()
for beta in betas:
    for k_p in keep_probs:
        with graph.as_default():

            # Input data. For the training data, we use a placeholder that will be fed
            # at run time with a training minibatch.
            tf_train_dataset = tf.placeholder(tf.float32,
                                            shape=(batch_size, image_size * image_size))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            tf_valid_dataset = tf.constant(valid_dataset)
            tf_test_dataset = tf.constant(test_dataset)
            keep_prob = tf.placeholder(tf.float32)

            # Variables.
            weights_h1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
            weights_out = tf.Variable(
              tf.truncated_normal([num_hidden_nodes, num_labels]))
            biases_in = tf.Variable(tf.zeros([num_hidden_nodes]))
            biases_out = tf.Variable(tf.zeros([num_labels]))

            # Training computation.
            layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_in)
            drop_out = tf.nn.dropout(layer_1, keep_prob=k_p)
            logits = tf.matmul(drop_out, weights_out) + biases_out

            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) +\
                beta*(tf.nn.l2_loss(weights_h1) + tf.nn.l2_loss(weights_out))

            # Optimizer.
            optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

            # Predictions for the training, validation, and test data.
            train_prediction = tf.nn.softmax(logits)
            valid_prediction = tf.nn.softmax(
              tf.matmul(
                  tf.nn.dropout(
                      tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_in)
                      , keep_prob=1)
                  , weights_out) + biases_out
            )
            test_prediction = tf.nn.softmax(
              tf.matmul(
                  tf.nn.dropout(
                      tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_in)
                      , keep_prob=1)
                  , weights_out) + biases_out
            )

        num_steps = 3001
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print("Initialized")
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_dataset_small[offset:(offset + batch_size), :]
                batch_labels = train_labels_small[offset:(offset + batch_size), :]
                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {tf_train_dataset : batch_data
                             , tf_train_labels : batch_labels
                            }
                _, l, predictions = session.run(
                  [optimizer, loss, train_prediction], feed_dict=feed_dict)
#                 if (step % 500 == 0):
#                     print("Minibatch loss at step %d: %f" % (step, l))
#                     print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#                     print("Validation accuracy: %.1f%%" % accuracy(
#                         valid_prediction.eval(), valid_labels))

            print("\nkeep prob: %.4f" % k_p)
            print("beta: %.4f" % beta)
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            test_accuracy_1_hl = accuracy(test_prediction.eval(), test_labels)
            print("Test accuracy: %.1f%%\n" % test_accuracy_1_hl)

Initialized

keep prob: 0.0100
beta: 0.0010
Validation accuracy: 10.0%
Test accuracy: 10.0%

Initialized

keep prob: 0.1000
beta: 0.0010
Validation accuracy: 68.6%
Test accuracy: 75.3%

Initialized

keep prob: 0.2500
beta: 0.0010
Validation accuracy: 70.6%
Test accuracy: 78.2%

Initialized

keep prob: 0.3500
beta: 0.0010
Validation accuracy: 70.3%
Test accuracy: 77.4%

Initialized

keep prob: 0.4000
beta: 0.0010
Validation accuracy: 69.2%
Test accuracy: 75.8%

Initialized

keep prob: 0.5000
beta: 0.0010
Validation accuracy: 69.7%
Test accuracy: 77.0%

Initialized

keep prob: 0.6000
beta: 0.0010
Validation accuracy: 69.0%
Test accuracy: 76.1%

Initialized

keep prob: 0.7500
beta: 0.0010
Validation accuracy: 67.5%
Test accuracy: 74.2%

Initialized

keep prob: 0.9000
beta: 0.0010
Validation accuracy: 64.3%
Test accuracy: 71.6%

Initialized

keep prob: 1.0000
beta: 0.0010
Validation accuracy: 61.8%
Test accuracy: 68.6%



---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
